# Dataset Clustering


### Dataset Description

Name : [Energy consumption of the Netherlands](https://www.kaggle.com/lucabasa/dutch-energy)

Enexis, Liander, and Stedin are the three major network administrators of the Netherlands and, together, they provide energy to nearly the entire country. Every year, they release on their websites a table with the energy consumption of the areas under their administration.

The data are anonymized by aggregating the Zipcodes so that every entry describes at least 10 connections.

This market is not competitive, meaning that the zones are assigned. This means that every year they roughly provide energy to the same zipcodes. Small changes can happen from year to year either for a change of management or for a different aggregation of zipcodes.

In [33]:
# Spark Initialization

from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [34]:
print(spark)

## Dataset Preprocess

### Combine all the dataset

combine dataset from Enexis, Liander, and Stedin and group by years

In [64]:
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from os.path import join, isfile
from os import path, scandir, listdir

import gc

In [65]:
def list_all_files(location='../dataset/', pattern=None, recursive=True):
    """
    This function returns a list of files at a given location (including subfolders)
    
    - location: path to the directory to be searched
    - pattern: part of the file name to be searched (ex. pattern='.csv' would return all the csv files)
    - recursive: boolean, if True the function calls itself for every subdirectory it finds
    """
    subdirectories= [f.path for f in scandir(location) if f.is_dir()]
    files = [join(location, f) for f in listdir(location) if isfile(join(location, f))]
    if recursive:
        for directory in subdirectories:
            files.extend(list_all_files(directory))
    if pattern:
        files = [f for f in files if pattern in f]
    return files

In [66]:
list_all_files('../dataset/Electricity/', pattern='stedin')

['../dataset/Electricity/stedin_electricity_2019.csv',
 '../dataset/Electricity/stedin_electricity_2018.csv',
 '../dataset/Electricity/stedin_electricity_2009.csv',
 '../dataset/Electricity/stedin_electricity_2010.csv',
 '../dataset/Electricity/stedin_electricity_2011.csv',
 '../dataset/Electricity/stedin_electricity_2013.csv',
 '../dataset/Electricity/stedin_electricity_2012.csv',
 '../dataset/Electricity/stedin_electricity_2016.csv',
 '../dataset/Electricity/stedin_electricity_2017.csv',
 '../dataset/Electricity/stedin_electricity_2015.csv',
 '../dataset/Electricity/stedin_electricity_2014.csv']

In [67]:
def importer(file_list):
    imported = {}
    for file in file_list:
        yr = file.split('_')[-1].split('.')[0]
        if '0101' in yr:
            yr = yr.replace('0101', '')
        name = file.split('/')[-1].split('_')[0]
        # print(name, yr)
        df = pd.read_csv(file)
        # print(df.shape)
        imported[name + '_' + yr] = df
        del df
    return imported

In [68]:
elec_list = list_all_files('../dataset/Electricity/')
gas_list = list_all_files('../dataset/Gas/')
imp_elec = importer(elec_list)
imp_gas = importer(gas_list)
print('Done!')

Done!


In [70]:
def merge_manager(data_dict):
    all_man = pd.DataFrame()
    n_rows = 0
    for key in data_dict.keys():
        df = data_dict[key].copy()
        yr = key.split('_')[1]
        yr = str(int(yr) - 1) # account for the "delayed data issue"
        df = df.rename(columns={'annual_consume' : 'annual_consume_' + yr,
                               'delivery_perc': 'delivery_perc_' + yr,
                               'num_connections': 'num_connections_' + yr,
                               'perc_of_active_connections': 'perc_of_active_connections_' + yr,
                               'annual_consume_lowtarif_perc': 'annual_consume_lowtarif_perc_' + yr,
                               'smartmeter_perc': 'smartmeter_perc_' + yr})
        del df['type_conn_perc']
        del df['type_of_connection']
        del df['net_manager']
        del df['purchase_area']
        n_rows += df.shape[0]
        if len(all_man) == 0:
            all_man = df.copy()
        else:
            del df['street']
            del df['city']
            all_man = pd.merge(all_man, df, on=['zipcode_from', 'zipcode_to'], how='inner') # 'city', 'street',  
        del df
        gc.collect()
    print(f"Total rows before merge: {n_rows}")
    print(f"Total rows after merge: {all_man.shape[0]}")
    return all_man


def merge_yr(data_dict):
    all_yr = pd.DataFrame()
    for manager in ['enexis', 'liander', 'stedin']:
        print(manager)
        tmp = { key: data_dict[key] for key in data_dict.keys() if manager in key}
        all_man = merge_manager(tmp)
        if len(all_yr) == 0:
            all_yr = all_man.copy()
        else:
            all_yr = pd.concat([all_yr, all_man], ignore_index=True, join='inner')
        del all_man
        gc.collect()
        print("_"*40)
    print(f"Final shape: {all_yr.shape}")
    return all_yr

In [71]:
print("Electricity merging...")
elec_full = merge_yr(imp_elec)
print('_'*40)
print('_'*40)
print("Gas merging...")
gas_full = merge_yr(imp_gas)

Electricity merging...
enexis
Total rows before merge: 1141402
Total rows after merge: 90692
________________________________________
liander
Total rows before merge: 1511295
Total rows after merge: 126360
________________________________________
stedin
Total rows before merge: 913096
Total rows after merge: 88948
________________________________________
Final shape: (306000, 64)
________________________________________
________________________________________
Gas merging...
enexis
Total rows before merge: 950202
Total rows after merge: 63425
________________________________________
liander
Total rows before merge: 1200607
Total rows after merge: 97952
________________________________________
stedin
Total rows before merge: 934948
Total rows after merge: 79621
________________________________________
Final shape: (240998, 64)


In [74]:
elec_full.head()

,street,zipcode_from,zipcode_to,city,delivery_perc_2015,num_connections_2015,perc_of_active_connections_2015,annual_consume_2015,annual_consume_lowtarif_perc_2015,smartmeter_perc_2015,...,perc_of_active_connections_2018,annual_consume_2018,annual_consume_lowtarif_perc_2018,smartmeter_perc_2018,num_connections_2017,delivery_perc_2017,perc_of_active_connections_2017,annual_consume_2017,annual_consume_lowtarif_perc_2017,smartmeter_perc_2017
0,Sasdijk,4251AB,4251AB,WERKENDAM,100.0,15,100.0,4728,29.0,7.0,...,100.0,4614.24,34.24,41.18,17,100.00,100.0,4149.24,28.33,29.41
1,Sasdijk,4251AC,4251AC,WERKENDAM,80.0,10,100.0,3908,33.0,20.0,...,100.0,2520.73,34.31,36.36,11,81.82,100.0,2389.27,34.60,36.36
2,Sasdijk,4251AD,4251AD,WERKENDAM,100.0,29,100.0,3850,30.0,14.0,...,100.0,4275.46,38.34,28.57,29,100.00,100.0,3866.21,32.11,17.24
3,Nieuweweg,4251AE,4251AG,WERKENDAM,95.0,21,100.0,4751,52.0,14.0,...,100.0,4620.24,60.18,33.33,21,90.48,100.0,4416.52,57.26,33.33
4,Koppenhof,4251AH,4251AH,WERKENDAM,100.0,13,100.0,3084,31.0,23.0,...,100.0,3037.38,40.24,38.46,13,100.00,100.0,3233.77,27.92,30.77


## Export to csv

In [75]:
elec_full.to_csv(r'./electricity.csv')

In [76]:
gas_full.to_csv(r'./gas.csv')

## Clustering Session

In [81]:
## reload dataset
df = spark.read.csv("./electricity.csv", header=True, inferSchema=True)

In [82]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- delivery_perc_2015: double (nullable = true)
 |-- num_connections_2015: integer (nullable = true)
 |-- perc_of_active_connections_2015: double (nullable = true)
 |-- annual_consume_2015: integer (nullable = true)
 |-- annual_consume_lowtarif_perc_2015: double (nullable = true)
 |-- smartmeter_perc_2015: double (nullable = true)
 |-- delivery_perc_2016: double (nullable = true)
 |-- num_connections_2016: integer (nullable = true)
 |-- perc_of_active_connections_2016: double (nullable = true)
 |-- annual_consume_2016: integer (nullable = true)
 |-- annual_consume_lowtarif_perc_2016: double (nullable = true)
 |-- smartmeter_perc_2016: double (nullable = true)
 |-- delivery_perc_2014: double (nullable = true)
 |-- num_connections_2014: integer (nullable = true)
 |-- perc_of_active_connec

In [86]:
# assembling vector
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["annual_consume_2009","annual_consume_2010","annual_consume_2011","annual_consume_2012","annual_consume_2013","annual_consume_2014","annual_consume_2015","annual_consume_2016","annual_consume_2017","annual_consume_2018"],
    outputCol='features')

data = assembler.transform(df)
data.show()

+---+----------------+------------+----------+---------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+----------------------

In [87]:
# Train model
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans = KMeans().setK(10).setSeed(1)
model = kmeans.fit(data)

In [88]:
# Make a prediction
predictions = model.transform(data)
predictions.show(5)

+---+---------+------------+----------+---------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-------------------------------+-------------------+---------------------------------+--------------------+------------------+--------------------+-----------------------------

In [89]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.5801493305258678


In [90]:
# Shows the result
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[3919.82004705 3910.12173288 3899.24731279 3834.66635357 3769.55037899
 3630.38734154 3536.35699817 3455.02060736 3385.44124804 3295.52559004]
[21618.91920474 21468.97631134 21610.42343486 21754.18401015
 21586.25465313 20897.84348562 20586.42174281 20129.46996616
 19670.34975888 19565.06972081]
[11487.5735749  11442.19907203 11560.01590809 11516.10903668
 11471.77242598 11109.42366328 10928.18603624 10761.69034468
 10643.60948188 10383.82007844]
[45831.45454545 45819.63636364 45635.85353535 39346.4040404
 38431.69191919 37555.8030303  37144.93939394 36092.33333333
 42031.75636364 35397.62439394]
[5586.0473095  5567.7352835  5582.07143824 5534.51426508 5445.32471109
 5246.59218129 5151.07441766 5028.02476074 4925.26178088 4813.89441563]
[2487.14652459 2482.38339226 2473.586992   2451.8755148  2420.78063564
 2343.4638958  2295.58295128 2261.49117064 2225.50444615 2190.40018057]
[29675.54152446 29703.89192264 29842.66894198 29837.54379977
 29438.98634812 28571.62571104 